In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Frame Bundle Geometry on Ellipsoids Embedded in $\mathbb R^3$

In [ ]:
from src.manifolds.ellipsoid import *
M = Ellipsoid(params=np.array([1,1,1]))
#M.params.set_value(np.array([1,0.5,0.5]))
print(M)

from src.plotting import *

In [ ]:
from src.Riemannian import metric
from src.framebundle import FM

metric.initialize(M)
FM.initialize(M)

x = np.array([0.,0.])

# element u=(x,nu) in FM, nu being frame for T_xM
# np.linalg.cholesky(M.gsharpf(x)) gives orthonormal basis for T_xM, multiplication scales in given directions
nu = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x)))
u = np.concatenate((x,nu.flatten()))

# FM covector p
v = np.array([1.,1.])
px = np.linalg.solve(nu,v) # manifold part
pu = np.array([0.,0.,0.,0.]) # frame part
p = np.concatenate([px,pu])

print("u = ", u)
print("p = ", p)

# Geodesic on Frame bundle:

In [ ]:
# test Hamiltionian dynamics on FM from sub-Riemannian structure <v,w>_FM=<u^-1(v),u^-1(w)>_R^2
print(M.H_FMf(u,p))

# compute FM geodesic
us = M.Exp_Hamiltonian_FMtf(u,p).T

# plot
newfig()
M.plot(rotate=(30,80))
M.plotFMx(us,N_vec=5,linewidth=1.5,s=50)
plt.show()

# dynamics returning both position and momentum
(ts,qps) = M.Hamiltonian_dynamics_FMf(u,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.H_FMf(q,p) for (q,p) in zip(us,ps)]))

# Development and Stochastic Development

In [ ]:
# development dynamics
from src.framebundle import FM
from src.stochastics import stochastic_development
FM.initialize(M)
stochastic_development.initialize(M)

In [ ]:
# deterministic development

# curve in R^2
t = np.linspace(0,10,n_steps.get_value()+1)
gamma = np.vstack([[20*np.sin(t), t**2 + 2*t]]).T
dgamma = np.diff(gamma, axis = 0)

(ts,us) = M.developmentf(u,dgamma)

# plot with frame
newfig()
M.plot()
M.plotFMx(us)
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plotx(us[:,0:M.dim.eval()])
plt.show()

In [ ]:
# stochastic development
w = dWsf(M.dim.eval()) # noise / anti-development
(ts,us) = M.stochastic_developmentf(u,w)

# plot with frame
newfig()
M.plot()
M.plotFMx(us)
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plotx(us[:,0:M.dim.eval()])
plt.show()

# plot noise / anti-development
plt.figure()
plt.plot(w[:,0],w[:,1])
plt.axis('equal')
plt.show()

# Anisotropic  Normal Distribution

In [ ]:
# plot sample data with trajectories
K = 8
obss = np.zeros((K,n_steps.eval(),M.dim.eval()))
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[:,0:M.dim.eval()]
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
newfig()
M.plot()
M.plotFMx(u)
for i in range(K):
    M.plotx(obss[i],linewidth=.5,color=colors[i])
plt.show()

In [ ]:
# sample data
K = 1024
obss = np.zeros((K,M.dim.eval()))
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[-1][0:M.dim.eval()]
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
newfig()
M.plot()
M.plotFMx(u)
for i in range(K):
    M.plotx(obss[i])
plt.show()

In [ ]:
# plot estimated density, 
newfig()
# plotM(alpha=.4)
# plot_sphere_density_estimate(M, np.array([M.Ff(obs) for obs in obss]),pts=100,alpha=.8,bandwidth=.15) # spherical coordinates
plot_density_estimate(M,np.array([M.Ff(obs) for obs in obss]),limits=[-3,3,-3,3],pts=500,alpha=.4,bandwidth=.15) # general ellipsoidal coordinates (note: very long computation time)
plt.show()

# Most Probable Pahts

In [ ]:
# Compute 'most probable path' (in the sense of the driving semi-martingale) between u and x2
x2 = np.array([0.5,-0.5])

# cotangent vector for the MPP:
px2 = M.Log_FM(u,x2)

# MPP from u to x2:
usv = M.Exp_Hamiltonian_FMf(u,px2).T

newfig()
ax = plt.gca(projection='3d')
M.plot(rotate=(30,-45))
M.plotFMx(usv,N_vec=5,linewidth=1.5,s=50)
xq = M.Ff(x2)
ax.scatter(xq[0],xq[1],xq[2],color='r')
plt.show()

# Horizontal Vector Fields

In [ ]:
def plotHorizontalFields(u,color='b',color_intensity=1.,linewidth=3.,prevx=None,last=True):
        xM = u[0:M.dim.eval()]
        if xM.shape[0] == M.dim.eval(): # map to manifold
            xM = M.Ff(xM)
         
        ax = plt.gca(projection='3d')
        ax.scatter(xM[0],xM[1],xM[2],color=color)

        x = u[0:M.dim.eval()]
        nu = u[M.dim.eval():].reshape((M.dim.eval(),-1))
        
        # plot frame and horizontal variation
        Hu = M.Horizontalf(u) # horizontal basis fields
        Hnu = Hu[M.dim.eval():].reshape((M.dim.eval(),nu.shape[1],nu.shape[1])) # nu part
        JFx = M.JFf(x)
        for j in range(M.dim.eval()):
            nujM = np.dot(JFx,nu[:,j])
            HnujM = np.dot(JFx,np.dot(Hnu,nu[:,j]))
            ax.quiver(xM[0],xM[1],xM[2],nujM[0],nujM[1],nujM[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=1,
                      color='black',normalize=True,length=np.linalg.norm(nujM))
            for k in range(nu.shape[1]):
                basep = xM + nujM
                ax.quiver(basep[0],basep[1],basep[2],
                          HnujM[0,k],HnujM[1,k],HnujM[2,k], pivot='tail',linewidths=2.,
                          color='red',normalize=True,length=0.3)

# plot horizontal fields at x = (.1,.1) (vanishes at north pole)
x = np.array([0.1,0.1])
nu = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x)))
u = np.concatenate((x,nu.flatten()))

newfig()
M.plot()
plotHorizontalFields(u)
plt.show()